In [6]:
pip install tensorflow-gpu

Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install transformers

  Using cached transformers-4.25.1-py3-none-any.whl (5.8 MB)
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
Note: you may need to restart the kernel to use updated packages.


In [80]:
pip install fastai

  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install openai

In [148]:


import tensorflow as tf

from keras import backend as K


from keras.callbacks import ModelCheckpoint,  CSVLogger
from keras.layers import Add, Dense, Input, LSTM
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

import numpy as np
import pandas as pd
import joblib
from transformers import RobertaTokenizer, RobertaForMaskedLM
from torch.utils.data import DataLoader
from transformers import get_scheduler
from transformers import AdamW
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
import torch
import random
from fastai.text.all import *
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments


[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Максим\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [46]:
import json

In [129]:
df=pd.read_excel("DatasetFinal.xlsx")
df=df.drop(['Unnamed: 0'], axis=1)

In [29]:
df

,FirstRow,SecondRow,ThirdRow,Original,FirstKeyPhrase,SecondKeyPhrase,KeyEmotion
0,delicate savage,you'll never hold the cinder,but still you will burn,delicate savage you'll never hold the cinder but still you will burn,never hold,delicate savage,POSITIVE
1,our destination,the skyline of this city,shining horizon,our destination the skyline of this city shining horizon,city shining horizon,skyline,POSITIVE
2,a splash and a cry,words pulled from the riverside,dried in the hot sun,a splash and a cry words pulled from the riverside dried in the hot sun,cry words pulled,riverside dried,POSITIVE
3,hurt but poised for war,sturdy in crestfallen slumps,warrior spirit,hurt but poised for war sturdy in crestfallen slumps warrior spirit,crestfallen slumps warrior spirit,war sturdy,NEGATIVE
4,steamy mist rising,rocks receiving downward crash,as the jungle weeps,steamy mist rising rocks receiving downward crash as the jungle weeps,steamy mist rising rocks receiving downward crash,jungle weeps,NEGATIVE
...,...,...,...,...,...,...,...
13666,finding someone who,has pure intentions and heart,is so difficult,finding someone who has pure intentions and heart is so difficult,pure intentions,finding someone,NEGATIVE
13667,everything changes,in a split second so just,enjoy the moment,everything changes in a split second so just enjoy the moment,split second,everything changes,POSITIVE
13668,just because you could,spell your child's name a weird way,doesn't mean you should,just because you could spell your child's name a weird way doesn't mean you should,weird way,could spell,NEGATIVE
13669,i feel like alex,kate is gonna admit their,true identity,i feel like alex kate is gonna admit their true identity,feel like alex kate,true identity,POSITIVE


In [130]:
Y=df[["Original"]]

In [132]:
newY=[]
for i in range(0,len(X)):
    newY.append(Y.iloc[i]['Original'])
Y=newY

In [140]:
with open('haiku.txt', 'w',encoding="utf-8") as outfile:
    for i in Y:
        outfile.write("%s\n" % i)

In [ ]:
#Сдесь начинается тренировка

In [149]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)
      
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

In [280]:
train_file_path = "haiku.txt"
model_name = 'gpt2'
output_dir = 'AAAModelresultFinal'
overwrite_output_dir = False
per_device_train_batch_size = 20
num_train_epochs = 25.0
save_steps = 500

In [281]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

loading file vocab.json from cache at C:\Users\Максим/.cache\huggingface\hub\models--gpt2\snapshots\e7da7f221d5bf496a48136c0cd264e630fe9fcc8\vocab.json
loading file merges.txt from cache at C:\Users\Максим/.cache\huggingface\hub\models--gpt2\snapshots\e7da7f221d5bf496a48136c0cd264e630fe9fcc8\merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at C:\Users\Максим/.cache\huggingface\hub\models--gpt2\snapshots\e7da7f221d5bf496a48136c0cd264e630fe9fcc8\config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n

Step,Training Loss
500,3.843500
1000,3.374200
1500,3.077000
2000,2.870400
2500,2.758100


Saving model checkpoint to AAAModelresultFinal\checkpoint-500
Configuration saved in AAAModelresultFinal\checkpoint-500\config.json
Model weights saved in AAAModelresultFinal\checkpoint-500\pytorch_model.bin
Saving model checkpoint to AAAModelresultFinal\checkpoint-1000
Configuration saved in AAAModelresultFinal\checkpoint-1000\config.json
Model weights saved in AAAModelresultFinal\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to AAAModelresultFinal\checkpoint-1500
Configuration saved in AAAModelresultFinal\checkpoint-1500\config.json
Model weights saved in AAAModelresultFinal\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to AAAModelresultFinal\checkpoint-2000
Configuration saved in AAAModelresultFinal\checkpoint-2000\config.json
Model weights saved in AAAModelresultFinal\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to AAAModelresultFinal\checkpoint-2500
Configuration saved in AAAModelresultFinal\checkpoint-2500\config.json
Model weights saved in AA